In [1]:
######################################################
# (0) 초기 설정
######################################################
import os, random, torch
import numpy as np

def set_seed(seed):
    """
    재현성(일관된 결과)을 위해 SEED 고정하는 함수
    """
    os.environ['PYTHONHASHSEED'] = str(seed) 
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 예시: SEED 123
set_seed(123)

In [2]:
######################################################
# (1) 라이브러리 임포트 및 기본 설정
######################################################
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report

# 디바이스 설정 (GPU 사용 가능 시 'cuda', 아니면 'cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
######################################################
# (2) 하이퍼파라미터 & 경로 설정
######################################################
batch_size = 64 # 메모리 보면서 조절하세요
epochs = 50        # 예시로 20 epoch
learning_rate = 0.001
num_classes = 300
weight_decay = 1e-4                                                           # 기존 코드에서 weight_decay 없음. 이건 사용 안할 예정

# (사용 환경에 맞게 꼭 수정)
train_path = "/home/student/workspace/data/train"
test_path  = "/test/final_exam/challenge/test"

# 체크포인트 저장 폴더 (폴더명만 변경)
checkpoint_dir = "./checkpoint_inception_v3_je_condition"
os.makedirs(checkpoint_dir, exist_ok=True)

# 결과 저장 시 모델 이름 표기를 위한 변수
model_name = "inception_v3"  # ★ 추가

In [4]:
######################################################
# (3) 데이터 변환 & 데이터셋/로더 정의
######################################################
# ★ Inception v3는 보통 299×299 입력 권장.
#   필요시 다른 해상도 쓰셔도 됩니다.
transform_train = transforms.Compose([                       # 기존 코드에서 512, 512 등 모든 걸 다 가져왔음
    transforms.Resize((512, 512)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Train / Test 데이터셋
full_train_dataset = datasets.ImageFolder(train_path, transform=transform_train)
test_dataset = datasets.ImageFolder(test_path,  transform=transform_test)

# train, valid 층화추출 여기서 부터 수정
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Subset, DataLoader
import numpy as np

# Dataset 전체의 클래스 레이블 가져오기
targets = [full_train_dataset.targets[i] for i in range(len(full_train_dataset))]

# train, valid 셋 클래스 비율을 맞추기 위한 층화추출
# 층화추출을 위한 StratifiedShuffleSplit 설정
split = StratifiedShuffleSplit(n_splits=1, test_size=0.05, random_state=42)  # train: 80%, valid: 20%

# train_index와 valid_index 생성
for train_index, valid_index in split.split(np.zeros(len(targets)), targets):
    train_indices = train_index
    valid_indices = valid_index

# Subset을 사용하여 Train과 Valid Dataset 생성
train_dataset = Subset(full_train_dataset, train_indices)
valid_dataset = Subset(full_train_dataset, valid_indices)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# train, valid 셋 클래스 비율을 맞추기 위한 층화추출 확인
print(pd.Series(full_train_dataset.targets).value_counts(normalize=True, ascending=True))
print()
print(pd.Series(train_dataset.dataset.targets).value_counts(normalize=True, ascending=True))
print()
print(pd.Series(valid_dataset.dataset.targets).value_counts(normalize=True, ascending=True))



""" 기존 split
# Validation split (예: 5%)
train_size = int(0.95 * len(full_train_dataset))
val_size   = len(full_train_dataset) - train_size
train_dataset, validation_dataset = random_split(full_train_dataset, [train_size, val_size])

# DataLoader
train_loader      = DataLoader(train_dataset,      batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader       = DataLoader(test_dataset,       batch_size=batch_size, shuffle=False)

print(f"Training samples:   {len(train_dataset)}")
print(f"Validation samples: {len(validation_dataset)}")
print(f"Test samples:       {len(test_dataset)}") """

Training samples:   89179
Validation samples: 4694
Test samples:       4178


In [5]:
######################################################
# (4) 모델 정의 (Inception v3, scratch) & (선택)체크포인트 로드
######################################################
# ★ Inception v3 불러오기 → aux_logits=False 권장

model = models.inception_v3(pretrained=False)    # 기존 코드에서 이렇게 정의해서 변경함.
# model.fc = nn.Linear(model.fc.
model.fc = nn.Linear(model.fc.in_features, 300, bias= True)
model = model.to(device)

# [원하는 체크포인트 로드 시 주석 해제 예시]
# ckpt_path = os.path.join(checkpoint_dir, "checkpoint_epoch_14.pth")
# model.load_state_dict(torch.load(ckpt_path))
# print(f"Loaded checkpoint: {ckpt_path}")


/home/student/anaconda3/envs/1_pytorch_2.5.1/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [6]:
######################################################
# (5) 옵티마이저, 스케줄러, 손실함수 정의
######################################################
criterion = nn.CrossEntropyLoss()

# ★ Adam → AdamW로 변경
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)              # 기존 코드에서 weight decay는 사용하지 않음. 제거함.

# 스케줄러 (CosineAnnealingLR 그대로)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs) # 스케줄러만 고정


In [7]:
######################################################
# (6) 학습 함수, 검증 함수, 테스트 함수
######################################################
def train_model(num_epochs=epochs):
    """
    전체 학습 루프:
    1. Train 모드로 돌면서 배치마다 Forward → Loss 계산 → Backward → Optimizer update
    2. Validation Loss 측정
    3. 체크포인트 저장 + 스케줄러 step
    """
    for epoch in range(num_epochs):
        model.train()
        running_loss  = 0.0
        total_batches = len(train_loader)

        print(f"Epoch {epoch+1}/{num_epochs}")
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # 10번째 배치마다(or 마지막 배치) 로그 출력
            if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == total_batches:
                print(f"  [Batch {batch_idx+1}/{total_batches}] Loss: {loss.item():.4f}")

        # Validation Loss 체크
        val_loss = validate_model()

        # 매 Epoch마다 체크포인트 저장
        ckpt_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), ckpt_path)
        print(f"Checkpoint saved: {ckpt_path}")

        # 스케줄러 step
        scheduler.step()

        print(f"==> Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {running_loss/total_batches:.4f} | "
              f"Val Loss: {val_loss:.4f} | "
              f"LR: {scheduler.get_last_lr()[0]:.6f}\n")

def validate_model():
    """
    Validation 데이터셋을 통해 현재 모델 상태의 Loss 평가
    """
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    return val_loss / len(validation_loader)

def test_model(model, data_loader):
    """
    Test/Validation 데이터셋을 평가하여 정확도(Accuracy) 측정
    혼동행렬, F1-score 계산 등을 위해 예측값/정답을 리스트로 저장
    """
    model.eval()
    correct    = 0
    total      = 0
    all_preds  = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total   += labels.size(0)
            correct += (predicted == labels).sum().item()

            # F1, 혼동행렬 등을 위해 저장
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = 100.0 * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    return all_labels, all_preds


In [8]:
######################################################
# (7) 학습
######################################################
train_model(epochs)
print("----- Training Finished -----")

Epoch 1/50
  [Batch 10/186] Loss: 5.6705
  [Batch 20/186] Loss: 5.4064
  [Batch 30/186] Loss: 5.0325
  [Batch 40/186] Loss: 4.8271
  [Batch 50/186] Loss: 4.6020
  [Batch 60/186] Loss: 4.6165
  [Batch 70/186] Loss: 4.3791
  [Batch 80/186] Loss: 4.1928
  [Batch 90/186] Loss: 4.1018
  [Batch 100/186] Loss: 4.0824
  [Batch 110/186] Loss: 3.8271
  [Batch 120/186] Loss: 3.7534
  [Batch 130/186] Loss: 3.7277
  [Batch 140/186] Loss: 3.5102
  [Batch 150/186] Loss: 3.5962
  [Batch 160/186] Loss: 3.3707
  [Batch 170/186] Loss: 3.3567
  [Batch 180/186] Loss: 3.3027
  [Batch 186/186] Loss: 3.2150
Checkpoint saved: ./checkpoint_inception_v3/checkpoint_epoch_1.pth
==> Epoch [1/50] Train Loss: 4.2090 | Val Loss: 3.2899 | LR: 0.000999

Epoch 2/50
  [Batch 10/186] Loss: 3.0222
  [Batch 20/186] Loss: 2.8528
  [Batch 30/186] Loss: 2.7838
  [Batch 40/186] Loss: 2.5894
  [Batch 50/186] Loss: 2.7220
  [Batch 60/186] Loss: 2.6948
  [Batch 70/186] Loss: 2.6259
  [Batch 80/186] Loss: 2.3935
  [Batch 90/186] Los

KeyboardInterrupt: 

In [ ]:
######################################################
# (8) 평가(Eval) & CSV 제출
######################################################
# (선택) 원하는 체크포인트 로드 (주석 해제 시 사용)
# custom_ckpt = os.path.join(checkpoint_dir, "checkpoint_epoch_14.pth")
# model.load_state_dict(torch.load(custom_ckpt))
# print(f"Loaded checkpoint: {custom_ckpt}")

print("[Step 8] Evaluating current model & Saving CSV...")
all_labels_main, all_preds_main = test_model(model, test_loader)

# 제출용 CSV (sample_submission.csv 기반)
submission_main = pd.read_csv('./sample_submission.csv')
submission_main['Label'] = all_preds_main
submission_main.to_csv('./submission_epochMain.csv', index=False)
print("Submission file saved as 'submission_epochMain.csv'.")

In [ ]:
######################################################
# (9) 혼동행렬 / F1-score 계산 및 저장 -> 혼동 행렬 부분 좀 더 수정할 필요가 있음. 가중치 들고와서 할 수 있도록. 이건 나중에 하면 됩니다.
######################################################
# 클래스 이름 (훈련 데이터 기준 가정)
class_names = [f"class_{i}" for i in range(num_classes)]

# (1) 혼동 행렬 생성 및 저장
cm = confusion_matrix(all_labels_main, all_preds_main)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, cmap='Blues', annot=False, fmt='d', cbar_kws={"label": "Frequency"})
plt.title(f"Confusion Matrix ({model_name} - {epochs} Epochs)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig(f"confusion_matrix_{model_name}_epochs{epochs}.png", dpi=200)
plt.close()
print(f"Confusion matrix saved as 'confusion_matrix_{model_name}_epochs{epochs}.png'")

# (2) Validation 데이터 기준 F1-score 계산 및 저장
val_labels, val_preds = test_model(model, validation_loader)  # Validation 데이터로 예측
report_dict = classification_report(val_labels, val_preds,
                                    target_names=class_names,
                                    output_dict=True)

# F1 보고서 DataFrame 생성
report_df = pd.DataFrame(report_dict).transpose()
report_path = f"val_f1_report_{model_name}_epochs{epochs}.csv"
report_df.to_csv(report_path, index=True)
print(f"Validation F1-score report saved as '{report_path}'")

# (3) F1-score 낮은 클래스 50개 선택 및 CSV 저장
class_report_only = report_df.iloc[:num_classes]  # 300개 클래스만 선택
class_report_only = class_report_only.sort_values(by="f1-score", ascending=True)
lowest_50 = class_report_only.head(50)
lowest_50_path = f"val_lowest_50_classes_{model_name}_epochs{epochs}.csv"
lowest_50.to_csv(lowest_50_path)
print(f"Lowest 50 F1-score classes (Validation) saved as '{lowest_50_path}'")

print("\n--- Lowest 50 Classes by F1-Score (Validation) ---")
print(lowest_50[["precision", "recall", "f1-score"]])

In [ ]:
######################################################
# (10) F1 낮은 클래스 폴더로부터 파인튜닝 -> 여기서 부턴 좀 더 고려해봐야합니다.
######################################################
def fine_tune_low_f1_classes_folder(
    data_root,
    transform_train,
    model,
    optimizer,
    scheduler,
    device,
    criterion,
    epochs,
    bs,
    ckpt_dir
):
    ds = datasets.ImageFolder(data_root, transform_train)
    dl = DataLoader(ds, batch_size=bs, shuffle=True)
    model.train()
    for e in range(epochs):
        total_loss = 0.0
        for x, y in dl:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        save_path = os.path.join(ckpt_dir, f"fine_tune_folder_epoch_{e+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"[Fine-Tune Folder] Epoch {e+1}/{epochs} | Loss: {total_loss/len(dl):.4f} | Saved: {save_path}")

print("[Step 10] Start fine-tuning on 'lowest_30_data'...")
fine_tune_low_f1_classes_folder(
    data_root="lowest_30_data",
    transform_train=transform_train,
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    criterion=criterion,
    epochs=2,                # 예: 2 epoch
    bs=64,                   # 배치 사이즈
    ckpt_dir=checkpoint_dir
)
print("[Step 10] Fine-tuning finished.")

In [ ]:
######################################################
# (11) 최종 평가 (원하면 CSV 생성)
######################################################
print("[Step 11] Testing after fine-tuning...")
all_labels_ft, all_preds_ft = test_model(model, test_loader)

# 파인튜닝 후 CSV 저장
submission_ft = pd.read_csv('./sample_submission.csv')
submission_ft['Label'] = all_preds_ft
submission_ft.to_csv('./submission_epochFineTune.csv', index=False)
print("Submission file saved as 'submission_epochFineTune.csv'.")
print("----- Done -----")